<hr style="border:2px solid #0281c9"> </hr>

<img align="left" alt="ESO Logo" src="http://archive.eso.org/i/esologo.png">  

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;">ESO Science Archive</h1> 
  <h2 style="color: #0281c9; font-weight: bold;">Jupyter Notebooks</h2>
</div>

<hr style="border:2px solid #0281c9"> </hr>

# **Query by Source Name or Position**

This notebook demonstrates how to use the `astroquery.eso` module to search for and download raw and reduced HAWK-I near-infrared observations from the ESO archive, specifically targeting Sgr A*, the supermassive black hole at the center of the Milky Way. It guides users through querying both the main ESO archive table and instrument-specific tables, filtering results based on observation parameters, and retrieving raw, unprocessed data for further calibration and analysis, enabling direct access to ESO’s extensive collection of observational datasets.

<hr style="border:2px solid #0281c9"> </hr>

# **Importing and basic usage of astroquery.eso**

In [1]:
import astroquery # import astroquery
print(f"astroquery version: {astroquery.__version__}") # check the version of astroquery

astroquery version: 0.4.11.dev10290


In [2]:
from astroquery.eso import Eso # import the ESO module from astroquery

In [3]:
eso = Eso() # create an instance of the ESO class 

# **Performing a Small Cone Search Around Sgr A***

To perform a small cone search of **10 arcseconds** around the source **Sgr A**, we first need to resolve the source name to obtain its **Right Ascension (RA)** and **Declination (Dec)** coordinates. This can be done using an online name resolver such as the ``SkyCoord.from_name()`` method from astropy.coordinates, which allows us to resolve object names into precise RA/Dec coordinates using online astronomical databases. This gives (ICRS): 

```python
ra = 266.41681663
dec = -29.00782497
```

In coordinate values of degrees in the ICRS (International Celestial Reference System) frame.

In [4]:
from astropy.coordinates import SkyCoord # import the SkyCoord class from the astropy.coordinates module
import astropy.units as u # import the astropy.units module

In [6]:
target = "Sgr A*" # set the target to Sgr A*
coords = SkyCoord.from_name(target) # create a SkyCoord object from the name of the source 
radius = 3 *u.arcmin # set the radius of the search to 3 arcminutes

print(f"Coordinates of Sgr A*: {coords}") # print the coordinates of Sgr A*

Coordinates of Sgr A*: <SkyCoord (ICRS): (ra, dec) in deg
    (266.41681663, -29.00782497)>


In [7]:
eso.maxrec = 3    # For this example we limit the number of records to 3

## **Reduced data cone search**

In [ ]:
table_reduced = eso.query_surveys(column_filters={"instrument_name": "HAWKI"},
                                cone_ra=coords.ra.value, 
                                cone_dec=coords.dec.value, 
                                cone_radius=radius.to("deg").value) # query the ESO archive for HAWKI data

table_reduced["target_name", "s_ra", "s_dec", "proposal_id", "instrument_name", "dp_id"] # print the first 3 rows of the table

target_name,s_ra,s_dec,proposal_id,instrument_name,dp_id
,deg,deg,,,
object,float64,float64,object,object,object
GCRT,266.306198,-30.136131,381.C-0186(A),HAWKI,ADP.2016-07-26T11:56:15.302
GCRT,266.306022,-30.136066,381.C-0186(A),HAWKI,ADP.2016-07-26T11:56:15.324
GCRT,266.306022,-30.136066,381.C-0186(A),HAWKI,ADP.2016-07-26T11:56:15.335


## **Raw data cone search**

Similar cone search functionality is also available through `query_instrument()` and `query_main()` by passing the same `cone_ra`, `cone_dec`, and `cone_radius` arguments. For example:

In [ ]:
table_raw = eso.query_instrument("HAWKI",
                            cone_ra=coords.ra.value, 
                            cone_dec=coords.dec.value, 
                            cone_radius=radius.to("deg").value) # query the ESO archive for HAWKI data

table_reduced["target_name", "s_ra", "s_dec", "proposal_id", "instrument_name", "dp_id"] # print the first 3 rows of the table

target_name,s_ra,s_dec,proposal_id,instrument_name,dp_id
,deg,deg,,,
object,float64,float64,object,object,object
GCRT,266.306198,-30.136131,381.C-0186(A),HAWKI,ADP.2016-07-26T11:56:15.302
GCRT,266.306022,-30.136066,381.C-0186(A),HAWKI,ADP.2016-07-26T11:56:15.324
GCRT,266.306022,-30.136066,381.C-0186(A),HAWKI,ADP.2016-07-26T11:56:15.335


In [ ]:
table_raw = eso.query_main("HAWKI",
                        cone_ra=coords.ra.value, 
                        cone_dec=coords.dec.value, 
                        cone_radius=radius.to("deg").value) # query the ESO archive for HAWKI data

table_raw["object", "ra", "dec", "prog_id", "instrument", "dp_id"] # print the first 3 rows of the table

object,ra,dec,prog_id,instrument,dp_id
,deg,deg,,,
object,float64,float64,object,object,object
GCRT,266.30619805,-30.13448,381.C-0186(A),HAWKI,HAWKI.2008-07-25T00:12:09.059
GCRT,266.37044194,-30.07898,381.C-0186(A),HAWKI,HAWKI.2008-07-25T00:14:09.186
GCRT,266.36666194,-30.08555,381.C-0186(A),HAWKI,HAWKI.2008-07-25T00:14:45.580


# **Downloading datasets from the archive**

The returned table has an ``dp_id`` column. It can be used to retrieve the datasets with ``retrieve_data()``.

The file names, returned in ``data_files``, points to the decompressed datasets (without the .Z extension) that have been locally downloaded. 

The default location (in the ``astropy`` cache) of the decompressed datasets can be adjusted by providing a destination keyword in the call to ``retrieve_data()``. By default, if a requested dataset is already found, it is not downloaded again from the archive. To force the retrieval of data that are present in the destination directory, use ``continuation=True`` in the call to ``retrieve_data()``.

In [ ]:
data_files = eso.retrieve_data(table_raw["dp_id"][0]) # download the first data product
# data_files = eso.retrieve_data(table['dp_id'][0], continuation=True) # force the download of all files even if present in the destination directory
# data_files = eso.retrieve_data(table['dp_id'][0], destination='./') # specify the destination directory for the download

INFO: Downloading datasets ... [astroquery.eso.core]
INFO: Downloading 1 files ... [astroquery.eso.core]
INFO: Downloading file 1/1 https://dataportal.eso.org/dataPortal/file/HARPS.2003-06-18T09:06:13.920 to /Users/abarnes/.astropy/cache/astroquery/Eso [astroquery.eso.core]
INFO: Found cached file /Users/abarnes/.astropy/cache/astroquery/Eso/HARPS.2003-06-18T09:06:13.920.fits [astroquery.eso.core]
INFO: Done! [astroquery.eso.core]


<hr style="border:2px solid #0281c9"> </hr>